# Aplikasi Klasifikasi Suara Buka/Tutup dengan Streamlit

Notebook ini berisi implementasi aplikasi Streamlit untuk klasifikasi suara "buka" dan "tutup" dengan fitur:
1. Upload file audio
2. Rekam suara langsung
3. Prediksi real-time menggunakan model yang telah dilatih

## 1) Persiapan dan Install Dependencies

In [38]:
# Install dependencies yang diperlukan (uncomment jika belum terinstall)
# !pip install streamlit tsfel librosa soundfile numpy pandas scikit-learn scipy
# !pip install streamlit-webrtc av pydub
# !pip install audio-recorder-streamlit

print("Dependencies ready!")

Dependencies ready!


## 2) Training Model dan Menyimpan Model

In [39]:
import numpy as np
import pandas as pd
import librosa
import tsfel
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import joblib
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

Libraries imported successfully!


In [40]:
# Fungsi untuk memuat dan preprocess audio
def load_and_preprocess_audio(file_path, sr=16000):
    """Memuat file audio, trim silence, dan normalisasi"""
    y, sample_rate = librosa.load(file_path, sr=sr, mono=True)
    y_trimmed, _ = librosa.effects.trim(y, top_db=25)
    
    if np.max(np.abs(y_trimmed)) > 0:
        y_trimmed = y_trimmed / np.max(np.abs(y_trimmed))
    
    return y_trimmed, sample_rate

def extract_features(y, sr, cfg):
    """Ekstraksi fitur TSFEL dari audio"""
    try:
        X = tsfel.time_series_features_extractor(cfg, y, fs=sr, verbose=0)
        return X.iloc[0].values
    except Exception as e:
        print(f"Error extracting features: {e}")
        return None

print("Helper functions defined!")

Helper functions defined!


In [41]:
# Load dataset dan train model
print("Loading dataset...")

BUKA_PATH = './voice_augmented/buka/'
TUTUP_PATH = './voice_augmented/tutup/'
TARGET_SR = 16000

# Konfigurasi TSFEL
cfg = tsfel.get_features_by_domain(["statistical"])
n_feats = tsfel.get_number_features(cfg)

# Load semua audio
audio_data = []
labels = []

# Load buka
buka_files = sorted([f for f in os.listdir(BUKA_PATH) if f.endswith('.wav')])
print(f"Loading {len(buka_files)} files from 'buka'...")
for file in buka_files:
    y, sr = load_and_preprocess_audio(os.path.join(BUKA_PATH, file), TARGET_SR)
    audio_data.append(y)
    labels.append(0)

# Load tutup
tutup_files = sorted([f for f in os.listdir(TUTUP_PATH) if f.endswith('.wav')])
print(f"Loading {len(tutup_files)} files from 'tutup'...")
for file in tutup_files:
    y, sr = load_and_preprocess_audio(os.path.join(TUTUP_PATH, file), TARGET_SR)
    audio_data.append(y)
    labels.append(1)

print(f"\nTotal audio loaded: {len(audio_data)}")
print(f"Buka: {labels.count(0)}, Tutup: {labels.count(1)}")

Loading dataset...
Loading 100 files from 'buka'...
Loading 100 files from 'tutup'...

Total audio loaded: 200
Buka: 100, Tutup: 100


In [42]:
# Ekstraksi fitur untuk semua audio
print("Extracting features...")

features_list = []
feature_names_from_tsfel = None

for i, y in enumerate(audio_data):
    try:
        # Ekstrak fitur dengan TSFEL
        X = tsfel.time_series_features_extractor(cfg, y, fs=TARGET_SR, verbose=0)
        
        # Simpan nama kolom dari iterasi pertama
        if feature_names_from_tsfel is None:
            feature_names_from_tsfel = list(X.columns)
        
        features_list.append(X.iloc[0].values)
    except Exception as e:
        print(f"Error on audio {i}: {e}")
        features_list.append(np.full(len(feature_names_from_tsfel) if feature_names_from_tsfel else n_feats, np.nan))
    
    if (i + 1) % 20 == 0:
        print(f"Progress: {i + 1}/{len(audio_data)}")

# Buat DataFrame dengan nama kolom yang benar
X_df = pd.DataFrame(features_list, columns=feature_names_from_tsfel)
X_df['label'] = labels

print(f"\nTotal features extracted: {len(feature_names_from_tsfel)}")
print(f"Sample feature names: {feature_names_from_tsfel[:5]}")

# Clean data
X_clean = X_df.replace([np.inf, -np.inf], np.nan)
y_labels = X_clean['label'].values
X_features = X_clean.drop(columns=['label'])

# Drop NaN columns - SIMPAN NAMA KOLOM YANG TERSISA
X_features = X_features.dropna(axis=1, how='any')
final_feature_names = list(X_features.columns)

print(f"\nFeatures shape after cleaning: {X_features.shape}")
print(f"Feature names after cleaning: {len(final_feature_names)}")
print(f"Sample final feature names: {final_feature_names[:5]}")
print(f"Labels distribution: Buka={sum(y_labels==0)}, Tutup={sum(y_labels==1)}")

Extracting features...
Progress: 20/200
Progress: 40/200
Progress: 60/200
Progress: 80/200
Progress: 100/200
Progress: 120/200
Progress: 140/200
Progress: 160/200
Progress: 180/200
Progress: 200/200

Total features extracted: 31
Sample feature names: ['0_Absolute energy', '0_Average power', '0_ECDF Percentile Count_0', '0_ECDF Percentile Count_1', '0_ECDF Percentile_0']

Features shape after cleaning: (200, 31)
Feature names after cleaning: 31
Sample final feature names: ['0_Absolute energy', '0_Average power', '0_ECDF Percentile Count_0', '0_ECDF Percentile Count_1', '0_ECDF Percentile_0']
Labels distribution: Buka=100, Tutup=100


In [43]:
# Train model
print("Training model...")

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X_features, y_labels, test_size=0.2, random_state=42, stratify=y_labels
)

# Standardize
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
model.fit(X_train_scaled, y_train)

# Evaluate
train_score = model.score(X_train_scaled, y_train)
test_score = model.score(X_test_scaled, y_test)

print(f"\nTraining accuracy: {train_score:.4f}")
print(f"Test accuracy: {test_score:.4f}")
print("\nModel training completed!")

Training model...

Training accuracy: 1.0000
Test accuracy: 0.8750

Model training completed!


In [44]:
# Simpan model, scaler, dan konfigurasi
print("Saving model artifacts...")

# Simpan model
joblib.dump(model, 'voice_classifier_model.pkl')
print("✓ Model saved: voice_classifier_model.pkl")

# Simpan scaler
joblib.dump(scaler, 'voice_scaler.pkl')
print("✓ Scaler saved: voice_scaler.pkl")

# Simpan feature names - GUNAKAN final_feature_names YANG SUDAH ADA
joblib.dump(final_feature_names, 'feature_names.pkl')
print(f"✓ Feature names saved: feature_names.pkl ({len(final_feature_names)} features)")
print(f"  Sample names: {final_feature_names[:3]}")

# Simpan konfigurasi TSFEL
joblib.dump(cfg, 'tsfel_config.pkl')
print("✓ TSFEL config saved: tsfel_config.pkl")

# Simpan metadata
metadata = {
    'target_sr': TARGET_SR,
    'n_features': len(final_feature_names),
    'train_accuracy': train_score,
    'test_accuracy': test_score,
    'label_map': {0: 'Buka', 1: 'Tutup'}
}
joblib.dump(metadata, 'model_metadata.pkl')
print("✓ Metadata saved: model_metadata.pkl")

print("\n" + "="*50)
print("All artifacts saved successfully!")
print("="*50)

Saving model artifacts...
✓ Model saved: voice_classifier_model.pkl
✓ Scaler saved: voice_scaler.pkl
✓ Feature names saved: feature_names.pkl (31 features)
  Sample names: ['0_Absolute energy', '0_Average power', '0_ECDF Percentile Count_0']
✓ TSFEL config saved: tsfel_config.pkl
✓ Metadata saved: model_metadata.pkl

All artifacts saved successfully!


## 3) Buat File Streamlit App (app.py)

In [46]:
%%writefile app.py
import streamlit as st
import numpy as np
import librosa
import tsfel
import joblib
import soundfile as sf
import tempfile
import os
from io import BytesIO
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Konfigurasi halaman
st.set_page_config(
    page_title="Klasifikasi Suara Buka/Tutup",
    page_icon="🎙️",
    layout="wide"
)

# Load model dan artifacts
@st.cache_resource
def load_model_artifacts():
    model = joblib.load('voice_classifier_model.pkl')
    scaler = joblib.load('voice_scaler.pkl')
    feature_names = joblib.load('feature_names.pkl')
    cfg = joblib.load('tsfel_config.pkl')
    metadata = joblib.load('model_metadata.pkl')
    return model, scaler, feature_names, cfg, metadata

# Fungsi preprocessing
def preprocess_audio(y, sr, target_sr=16000):
    """Preprocess audio: resample, trim, normalize"""
    # Resample jika perlu
    if sr != target_sr:
        y = librosa.resample(y, orig_sr=sr, target_sr=target_sr)
        sr = target_sr
    
    # Trim silence
    y_trimmed, _ = librosa.effects.trim(y, top_db=25)
    
    # Normalize
    if np.max(np.abs(y_trimmed)) > 0:
        y_trimmed = y_trimmed / np.max(np.abs(y_trimmed))
    
    return y_trimmed, sr

# Fungsi ekstraksi fitur
def extract_features(y, sr, cfg, feature_names):
    """Ekstrak fitur TSFEL dari audio"""
    try:
        X = tsfel.time_series_features_extractor(cfg, y, fs=sr, verbose=0)
        X_clean = X.replace([np.inf, -np.inf], np.nan)
        
        # Ambil hanya fitur yang digunakan saat training
        features = []
        for fname in feature_names:
            if fname in X_clean.columns:
                features.append(X_clean[fname].values[0])
            else:
                features.append(np.nan)
        
        return np.array(features).reshape(1, -1)
    except Exception as e:
        st.error(f"Error extracting features: {str(e)}")
        return None

# Fungsi prediksi
def predict_audio(y, sr, model, scaler, cfg, feature_names, metadata):
    """Prediksi kelas audio"""
    # Preprocess
    y_proc, sr_proc = preprocess_audio(y, sr, metadata['target_sr'])
    
    # Extract features
    features = extract_features(y_proc, sr_proc, cfg, feature_names)
    
    if features is None:
        return None, None, None
    
    # Check for NaN
    if np.isnan(features).any():
        st.warning("Some features are NaN. Using mean imputation.")
        features = np.nan_to_num(features, nan=0.0)
    
    # Scale and predict
    features_scaled = scaler.transform(features)
    prediction = model.predict(features_scaled)[0]
    probabilities = model.predict_proba(features_scaled)[0]
    
    return prediction, probabilities, y_proc

# Fungsi visualisasi
def plot_waveform_and_spectrogram(y, sr):
    """Plot waveform dan spectrogram"""
    fig, axes = plt.subplots(2, 1, figsize=(10, 6))
    
    # Waveform
    librosa.display.waveshow(y, sr=sr, ax=axes[0])
    axes[0].set_title('Waveform')
    axes[0].set_xlabel('Waktu (detik)')
    axes[0].set_ylabel('Amplitudo')
    axes[0].grid(True, alpha=0.3)
    
    # Spectrogram
    S = librosa.amplitude_to_db(np.abs(librosa.stft(y, n_fft=1024, hop_length=256)), ref=np.max)
    img = librosa.display.specshow(S, sr=sr, hop_length=256, x_axis='time', y_axis='linear', ax=axes[1])
    axes[1].set_title('Spectrogram')
    fig.colorbar(img, ax=axes[1], format='%+2.0f dB')
    
    plt.tight_layout()
    return fig

# Main app
def main():
    # Header
    st.title("🎙️ Klasifikasi Suara Buka/Tutup")
    st.markdown("""
    Aplikasi ini mengklasifikasikan suara menjadi **Buka** atau **Tutup** menggunakan machine learning.
    Anda dapat mengupload file audio atau merekam suara secara langsung.
    """)
    
    # Load model
    with st.spinner('Loading model...'):
        model, scaler, feature_names, cfg, metadata = load_model_artifacts()
    
    # Sidebar - Info Model
    st.sidebar.header("📊 Informasi Model")
    st.sidebar.write(f"**Akurasi Training:** {metadata['train_accuracy']:.2%}")
    st.sidebar.write(f"**Akurasi Testing:** {metadata['test_accuracy']:.2%}")
    st.sidebar.write(f"**Jumlah Fitur:** {metadata['n_features']}")
    st.sidebar.write(f"**Sample Rate:** {metadata['target_sr']} Hz")
    st.sidebar.markdown("---")
    st.sidebar.header("ℹ️ Petunjuk Penggunaan")
    st.sidebar.markdown("""
    1. Pilih metode input (Upload atau Rekam)
    2. Upload file audio (.wav, .mp3) atau rekam suara
    3. Klik tombol **Prediksi**
    4. Lihat hasil prediksi dan visualisasi
    """)
    
    # Main content
    tab1, tab2 = st.tabs(["📁 Upload File Audio", "🎤 Rekam Suara"])
    
    # Tab 1: Upload File
    with tab1:
        st.header("Upload File Audio")
        uploaded_file = st.file_uploader(
            "Pilih file audio (WAV atau MP3)",
            type=['wav', 'mp3'],
            help="Upload file audio dengan format .wav atau .mp3"
        )
        
        if uploaded_file is not None:
            # Save to temporary file
            with tempfile.NamedTemporaryFile(delete=False, suffix=os.path.splitext(uploaded_file.name)[1]) as tmp_file:
                tmp_file.write(uploaded_file.read())
                tmp_path = tmp_file.name
            
            try:
                # Load audio
                y, sr = librosa.load(tmp_path, sr=None, mono=True)
                
                # Display audio player
                st.audio(uploaded_file, format=f'audio/{uploaded_file.name.split(".")[-1]}')
                
                col1, col2, col3 = st.columns([1, 1, 2])
                with col1:
                    st.metric("Durasi", f"{len(y)/sr:.2f} detik")
                with col2:
                    st.metric("Sample Rate", f"{sr} Hz")
                
                # Predict button
                if st.button("🔍 Prediksi", key="predict_upload", type="primary"):
                    with st.spinner('Memproses audio...'):
                        prediction, probabilities, y_proc = predict_audio(
                            y, sr, model, scaler, cfg, feature_names, metadata
                        )
                        
                        if prediction is not None:
                            # Display results
                            st.success("✅ Prediksi selesai!")
                            
                            # Prediction result
                            result_label = metadata['label_map'][prediction]
                            confidence = probabilities[prediction] * 100
                            
                            col1, col2 = st.columns(2)
                            
                            with col1:
                                st.markdown("### 🎯 Hasil Prediksi")
                                st.markdown(f"## **{result_label}**")
                                st.markdown(f"Confidence: **{confidence:.1f}%**")
                                
                                # Probability bars
                                st.markdown("#### Probabilitas:")
                                for i, label in metadata['label_map'].items():
                                    prob = probabilities[i] * 100
                                    st.progress(probabilities[i])
                                    st.write(f"{label}: {prob:.1f}%")
                            
                            with col2:
                                # Visualizations
                                st.markdown("### 📊 Visualisasi Audio")
                                fig = plot_waveform_and_spectrogram(y_proc, metadata['target_sr'])
                                st.pyplot(fig)
                                plt.close()
                
            finally:
                # Clean up temp file
                if os.path.exists(tmp_path):
                    os.unlink(tmp_path)
    
    # Tab 2: Record Audio
    with tab2:
        st.header("Rekam Suara Langsung")
        st.info("💡 **Tips:** Pastikan mikrofon Anda aktif dan izinkan akses browser ke mikrofon.")
        
        try:
            from audio_recorder_streamlit import audio_recorder
            
            # Audio recorder
            audio_bytes = audio_recorder(
                text="Klik untuk mulai/berhenti merekam",
                recording_color="#e74c3c",
                neutral_color="#3498db",
                icon_name="microphone",
                icon_size="3x"
            )
            
            if audio_bytes:
                # Save to temporary file
                with tempfile.NamedTemporaryFile(delete=False, suffix='.wav') as tmp_file:
                    tmp_file.write(audio_bytes)
                    tmp_path = tmp_file.name
                
                try:
                    # Load audio
                    y, sr = librosa.load(tmp_path, sr=None, mono=True)
                    
                    # Display audio player
                    st.audio(audio_bytes, format='audio/wav')
                    
                    col1, col2 = st.columns(2)
                    with col1:
                        st.metric("Durasi", f"{len(y)/sr:.2f} detik")
                    with col2:
                        st.metric("Sample Rate", f"{sr} Hz")
                    
                    # Predict button
                    if st.button("🔍 Prediksi", key="predict_record", type="primary"):
                        with st.spinner('Memproses audio...'):
                            prediction, probabilities, y_proc = predict_audio(
                                y, sr, model, scaler, cfg, feature_names, metadata
                            )
                            
                            if prediction is not None:
                                # Display results
                                st.success("✅ Prediksi selesai!")
                                
                                # Prediction result
                                result_label = metadata['label_map'][prediction]
                                confidence = probabilities[prediction] * 100
                                
                                col1, col2 = st.columns(2)
                                
                                with col1:
                                    st.markdown("### 🎯 Hasil Prediksi")
                                    st.markdown(f"## **{result_label}**")
                                    st.markdown(f"Confidence: **{confidence:.1f}%**")
                                    
                                    # Probability bars
                                    st.markdown("#### Probabilitas:")
                                    for i, label in metadata['label_map'].items():
                                        prob = probabilities[i] * 100
                                        st.progress(probabilities[i])
                                        st.write(f"{label}: {prob:.1f}%")
                                
                                with col2:
                                    # Visualizations
                                    st.markdown("### 📊 Visualisasi Audio")
                                    fig = plot_waveform_and_spectrogram(y_proc, metadata['target_sr'])
                                    st.pyplot(fig)
                                    plt.close()
                    
                finally:
                    # Clean up temp file
                    if os.path.exists(tmp_path):
                        os.unlink(tmp_path)
        
        except ImportError:
            st.warning("⚠️ Package 'audio-recorder-streamlit' belum terinstall.")
            st.code("pip install audio-recorder-streamlit", language="bash")
            st.info("Sebagai alternatif, Anda dapat menggunakan fitur Upload File Audio.")

if __name__ == "__main__":
    main()

Overwriting app.py


## 4) Jalankan Aplikasi Streamlit

Untuk menjalankan aplikasi, gunakan perintah berikut ini terminal:

```bash
streamlit run app.py
```